# Evaluación Modular 6:
# Segmentación y detección de anomalías en pacientes crónicos

## Objetivo
Aplicar técnicas avanzadas de aprendizaje no supervisado para identificar grupos de pacientes con condiciones clínicas similares, visualizar dichos segmentos en espacios de baja dimensión, detectar perfiles atípicos y analizar comparativamente distintos métodos de agrupamiento. Los estudiantes implementarán modelos de clustering como K-Means, DBSCAN o jerárquicos, emplearán técnicas de reducción de dimensionalidad como PCA o t-SNE para representar los resultados, y justificarán sus decisiones en función del contexto clínico y los resultados obtenidos.

**Datasets utilizados:**  
`Diabetes`

---

### Estructura del Notebook:
1. Metodología.
2. Configuración del entorno.
3. Definicion de funciones.
4. Uso de funciones y resultados.
5. Análisis de los resultados y reflexiones finales.

---

## 1. Metodología

### Flujo de trabajo

1. **Carga, preprocesamiento de datos y aplicación de PCA exploratorio:**
    - Se carga el dataset **Diabetes** y se escala.
    - Se aplica PCA para obtener la varianza acumulada y ver como varía con cada componente.

2. **Reducción de dimensionalidad, clustering y detección de anomalías:**
    - Se aplica PCA y UMAP al dataset escalado con 2 componentes (solo para visualización) y se aplica de nuevo con más componentes para el posterior clustering.
    - Se aplica DBSCAN y HDBSCAN cada uno con busqueda de hiperparametros como eps para DB y min_cluster_size para DBH.
    - Se aplica Isolation Forest y One-Class SVM a los modelos de DB y DBH con PCA y UMAP para detectar anomalías.

3. **Visualización e interpretación:**
    - Gráficos PCA y UMAP para los datos escalados, utilizando solo 2 componentes.
    - Gráficos de k-distance para identificar los mejores eps para DBSCAN para los modelos con PCA y UMAP con mas de 2 componentes.
    - Resultados de los mejores eps para DBSCAN y min_cluster_size para HDBSCAN.
    - Tabla comparativa con métricas de Silhouette, Calinski-Harabasz y Davies-Bouldin para los modelos DBSCAN y HDBSCAN con PCA y UMAP cada uno.
    - Gráficos de clusters para los modelos DBSCAN y HDBSCAN con PCA Y UMAP.
    - Resultados de las anomalías por modelo, índices de anomalías que se repiden en mas de un modelo, tabla con todas las anomalías detectadas por todos los modelos y gráficos de anomalías de Isolation Forest y One-Class SVM con PCA y UMAP.

---

# 2. Configuración del entorno

--- 

In [ ]:
# Librerías
import warnings
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import umap
import optuna
import hdbscan
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM

# Manejo de warnings y logs
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
optuna.logging.set_verbosity(logging.WARNING)

# 3. Definición de funciones

> **Nota:** Para mejor comprensión de las funciones y su utilidad, esta sección se divide en bloques, en donde cada uno responde a una parte diferente de la metodología de trabajo. 

---

**Bloque 1:** Carga, preprocesamiento de datos y aplicación de reducción de dimensionalidad de los datos.

- **`carga_y_preprocesamiento()`** 
Carga el dataset Diabetes, realiza escalado y prepara los datos para el análisis.

- **`aplicar_reducción()`** 
Aplica PCA y UMAP a los datos escalados 2 veces:
    - Primero genera un PCA y UMAP con 2 componentes solo para visualización inicial de los datos.
    - Despues se hace nuevamente un PCA y UMAP pero con mas componentes. Estos datos son los que se usarán en el flujo del proceso.

---

##### `Decisiones de diseño`

##### **Reducción para visualización (2D):**

Se usan PCA y UMAP con 2 componentes para proyectar los datos en un espacio bidimensional fácilmente interpretable y visualizable. Esto permite explorar la estructura general de los datos y observar posibles agrupamientos o separaciones de forma intuitiva mediante gráficos.

##### **Reducción para análisis y modelado (más componentes):**

- Para tareas de clustering y detección de anomalías, conservar una mayor cantidad de información es fundamental para evitar pérdidas importantes que podrían afectar la calidad de los resultados. Por ello:

    - En PCA, seleccionamos un umbral del 90% de varianza explicada (n_components=0.90), que indica que mantenemos las componentes suficientes para retener el 90% de la información original, balanceando reducción dimensional con preservación de la estructura de los datos.

    - En UMAP, optamos por un espacio con 10 componentes para captar complejidades no lineales presentes en los datos, ya que UMAP permite preservar relaciones topológicas más complejas en dimensiones más altas. Este número suele ser un buen punto medio para conservar información útil sin caer en la maldición de la dimensionalidad.

- De esta forma, se garantiza que la visualización sea clara y sencilla (2D) mientras que el análisis y modelado se realizan con una representación rica y representativa de los datos, maximizando la calidad de los clusters y la detección de anomalías.

---

In [ ]:
def cargar_y_preprocesar(path="diabetes.csv"):
    """
    Carga el dataset de diabetes desde un archivo CSV y aplica escalado estándar a las variables predictoras.

    Parámetros
    ----------
    path : str, opcional
        Ruta al archivo CSV que contiene el dataset. Por defecto es "diabetes.csv".

    Retorna
    -------
    X_scaled : np.ndarray
        Matriz de características escaladas.
    y : pd.Series
        Vector objetivo con las etiquetas de clase (Outcome).
    """
    df = pd.read_csv(path)
    X = df.drop(columns=["Outcome"])
    y = df["Outcome"]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, y

def aplicar_reduccion(X_scaled):
    """
    Aplica técnicas de reducción de dimensionalidad (PCA y UMAP) para visualización 2D y análisis posterior.

    Realiza:
    - PCA y UMAP en 2D para visualización de datos.
    - PCA reteniendo el 90% de varianza y UMAP con 10 componentes para análisis posterior (clustering y detección de anomalías).

    Parámetros
    ----------
    X_scaled : np.ndarray
        Matriz de características previamente escaladas.

    Retorna
    -------
    X_pca : np.ndarray
        Representación 2D de los datos usando PCA.
    X_umap : np.ndarray
        Representación 2D de los datos usando UMAP.
    X_pca_final : np.ndarray
        Representación PCA conservando el 90% de varianza.
    X_umap_final : np.ndarray
        Representación UMAP con 10 dimensiones y min_dist=0.1.
    """
    # PCA y UMAP para visualización (2D)
    pca = PCA(n_components=2, random_state=42)
    X_pca = pca.fit_transform(X_scaled)
    
    reducer = umap.UMAP(n_components=2, random_state=42)
    X_umap = reducer.fit_transform(X_scaled)
    
    # PCA y UMAP para análisis final (más componentes)
    pca_final = PCA(n_components=0.90, random_state=42)
    X_pca_final = pca_final.fit_transform(X_scaled)
    
    umap_final = umap.UMAP(n_components=10, min_dist=0.1, random_state=42)
    X_umap_final = umap_final.fit_transform(X_scaled)
    
    return X_pca, X_umap, X_pca_final, X_umap_final

**Bloque 2:** Clustering con DBSCAN y HDBSCAN.

- **`graficar_k_distance()`** 
Grafica la curva k-distancia para ayudar a elegir el parámetro eps en DBSCAN.

- **`busqueda_dbscan_eps()`** 
Realiza búsqueda y evaluación de diferentes valores de eps para DBSCAN con métricas de calidad.

- **`evaluar_dbscan_clustering()`** 
Evalúa el clustering DBSCAN con parámetros dados y entrega métricas resumidas en tabla.

- **`busqueda_hdbscan_min_cluster()`** 
Evalúa el rendimiento de HDBSCAN variando el parámetro min_cluster_size usando métricas de clustering.

- **`evaluar_hdbscan_clustering()`** 
Evalúa clustering HDBSCAN con parámetros específicos y entrega resultados en formato tabla.

---

##### `Decisiones de diseño`

##### **Justificación del uso de PCA y UMAP para comparación de modelos de clustering:**

- Para evaluar y comparar el desempeño de los modelos de clustering (DBSCAN y HDBSCAN), decidimos utilizar dos técnicas de reducción dimensional: PCA y UMAP. La razón principal es que cada técnica tiene características distintas en cómo preserva la estructura de los datos:

    - PCA es una técnica lineal que maximiza la varianza explicada y puede ser más adecuada para datos con relaciones lineales predominantes.

    - UMAP es una técnica no lineal que preserva tanto la estructura global como local del espacio de alta dimensión, permitiendo capturar relaciones más complejas y topologías no lineales.

- Comparar los resultados de clustering en ambos espacios nos permite validar la robustez y la sensibilidad de los modelos a la representación de los datos, asegurando que los clusters encontrados no sean artefactos de una sola técnica de reducción dimensional.

##### **Justificación del uso de múltiples métricas de evaluación de clustering:**

- Aunque el enunciado solicitaba principalmente el Índice de Silueta y Davies-Bouldin para evaluar los modelos, decidimos incluir también la métrica de Calinski-Harabasz para obtener una visión más completa y robusta de la calidad del clustering.

    - Índice de Silueta: Evalúa qué tan similar es un objeto a su propio cluster comparado con otros clusters; valores más altos indican clusters mejor definidos.

    - Davies-Bouldin: Mide la relación entre la dispersión intra-cluster y la separación entre clusters; valores más bajos son mejores.

    - Calinski-Harabasz: Considera la razón entre la varianza entre clusters y la varianza dentro de clusters; valores más altos indican clusters más compactos y bien separados.

- El uso conjunto de estas tres métricas ayuda a equilibrar diferentes aspectos de la calidad del clustering y reduce el riesgo de elegir un modelo subóptimo si solo se usara una métrica.

##### **Justificación del uso del gráfico de k-distance y de la elección cuidadosa del parámetro eps en DBSCAN:**

- Para el modelo DBSCAN, uno de los parámetros más importantes es eps, que define el radio máximo de vecindad para considerar que dos puntos están conectados. Una mala elección de este valor puede provocar dos errores críticos:

    - Si eps es demasiado pequeño, muchos puntos serán considerados ruido (outliers), y el modelo no encontrará clusters significativos.

    - Si eps es demasiado grande, los clusters se fusionarán y se perderá la estructura interna de los datos.

- Para mitigar esto, utilizamos el gráfico de k-distancia, una herramienta gráfica que permite estimar un valor adecuado de eps basándose en la distancia al k-ésimo vecino más cercano de cada punto. En este caso, usamos k = min_samples, y graficamos las distancias ordenadas. El punto de "codo" (el punto de mayor curvatura) en esta gráfica sugiere un valor razonable de eps, ya que representa una transición natural entre distancias intra-cluster y distancias entre ruido y densidad baja.

- Este enfoque permite una estimación visual e informada de eps, lo que mejora notablemente la estabilidad y calidad del clustering con DBSCAN.

##### **Justificación para la búsqueda de mejores valores de min_cluster_size en HDBSCAN:**

- En el caso de HDBSCAN, la selección del parámetro min_cluster_size es crítica para el desempeño del algoritmo, ya que controla el tamaño mínimo permitido para un cluster, afectando la granularidad y la sensibilidad a outliers.

- Optamos por realizar una búsqueda en un rango amplio (de 5 a 50 en pasos de 5) para este parámetro porque, sin esta exploración, estábamos "a ciegas", lo que resultaba en resultados pobres o poco representativos, incluso utilizando HDBSCAN que es considerado un algoritmo robusto. La búsqueda nos permitió encontrar valores de min_cluster_size que generan clusters significativos y con mejores métricas de evaluación.

##### **Justificación de mantener fijo el parámetro min_samples en DBSCAN y HDBSCAN:**

- El parámetro min_samples influye en la densidad mínima necesaria para considerar un punto como núcleo de cluster. Aunque ajustar min_samples podría mejorar los resultados, decidimos no optimizarlo debido a:

    - La complejidad y el tiempo de computación adicional que implicaría realizar una búsqueda conjunta de múltiples parámetros.

    - El hecho de que min_samples=4 es un valor comúnmente recomendado en la literatura para conjuntos de datos similares y funciona razonablemente bien con los valores explorados para eps y min_cluster_size.

    - Mantenerlo fijo facilita la comparación directa entre DBSCAN y HDBSCAN y simplifica la interpretación de resultados.

---

In [ ]:
# Se incluye aca la visualización ya que no es un fin, si no un medio para encontrar un rango de eps para probar
def graficar_k_distance(X_pca_final, X_umap_final, k=5):
    """
    Genera curvas k-distancia para estimar el valor óptimo de `eps` en DBSCAN usando datos reducidos con PCA y UMAP.

    Parámetros
    ----------
    X_pca_final : np.ndarray
        Datos transformados con PCA (varianza retenida).
    X_umap_final : np.ndarray
        Datos transformados con UMAP (componentes para análisis).
    k : int, opcional
        Número de vecinos a considerar para calcular la k-ésima distancia. Por defecto es 5.
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    for i, (X, titulo) in enumerate(zip([X_pca_final, X_umap_final], ["PCA", "UMAP"])):
        nbrs = NearestNeighbors(n_neighbors=k).fit(X)
        distances, _ = nbrs.kneighbors(X)
        distances = np.sort(distances[:, k-1])
        axes[i].plot(distances)
        axes[i].set_xlabel("Puntos ordenados")
        axes[i].set_ylabel(f"{k}-ésima distancia más cercana")
        axes[i].set_title(f"Curva k-distancia para {titulo}")
        axes[i].grid(True)
    plt.tight_layout()
    plt.show()

def busqueda_dbscan_eps(X, eps_values, min_samples=4, mostrar_resultados=True):
    """
    Ejecuta DBSCAN sobre una serie de valores `eps` y evalúa cada configuración con métricas de calidad de clustering.

    Parámetros
    ----------
    X : np.ndarray
        Datos de entrada para clustering.
    eps_values : list of float
        Lista de valores `eps` a evaluar.
    min_samples : int, opcional
        Número mínimo de muestras para formar un cluster. Por defecto es 4.
    mostrar_resultados : bool, opcional
        Si True, imprime los resultados por consola. Por defecto es True.

    Retorna
    -------
    resultados : list of dict
        Lista de diccionarios con métricas y etiquetas para cada valor de `eps`.
    """
    resultados = []
    
    for eps in eps_values:
        db = DBSCAN(eps=eps, min_samples=min_samples).fit(X)
        labels = db.labels_
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        
        if n_clusters > 1:
            sil_score = silhouette_score(X, labels)
            db_score = davies_bouldin_score(X, labels)
        else:
            sil_score = None
            db_score = None
        
        resultados.append({
            'eps': eps,
            'n_clusters': n_clusters,
            'silhouette': sil_score,
            'db_index': db_score,
            'labels': labels
        })
    
    if mostrar_resultados:
        print("\nResultados de evaluación de valores eps:")
        for r in resultados:
            resumen = {k: v for k, v in r.items() if k != 'labels'}
            print(resumen)
    
    return resultados

def evaluar_dbscan_clustering(X_pca, X_umap, min_samples=4, eps_pca=2.4, eps_umap=1.0):
    """
    Aplica DBSCAN con parámetros fijos sobre datos PCA y UMAP, y evalúa el rendimiento del clustering.

    Parámetros
    ----------
    X_pca : np.ndarray
        Datos transformados con PCA (2D).
    X_umap : np.ndarray
        Datos transformados con UMAP (2D).
    min_samples : int, opcional
        Valor de min_samples para DBSCAN. Por defecto es 4.
    eps_pca : float, opcional
        Valor de eps para aplicar DBSCAN sobre PCA. Por defecto es 2.4.
    eps_umap : float, opcional
        Valor de eps para aplicar DBSCAN sobre UMAP. Por defecto es 1.0.

    Retorna
    -------
    df_resultados : pd.DataFrame
        Tabla con métricas de evaluación para cada técnica de reducción.
    labels_pca : np.ndarray
        Etiquetas resultantes del clustering sobre PCA.
    labels_umap : np.ndarray
        Etiquetas resultantes del clustering sobre UMAP.
    """
    resultados = []
    all_labels = {}

    for nombre, X, eps in [("PCA", X_pca, eps_pca), ("UMAP", X_umap, eps_umap)]:
        model = DBSCAN(eps=eps, min_samples=min_samples).fit(X)
        labels = model.labels_
        all_labels[nombre] = labels  # Guarda los labels

        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

        if n_clusters >= 2:
            silhouette = silhouette_score(X, labels)
            ch_score = calinski_harabasz_score(X, labels)
            db_score = davies_bouldin_score(X, labels)
        else:
            silhouette = ch_score = db_score = float("nan")

        resultados.append({
            "Técnica": nombre,
            "Modelo": "DBSCAN",
            "Clusters": n_clusters,
            "Silhouette": silhouette,
            "Calinski-Harabasz": ch_score,
            "Davies-Bouldin": db_score
        })

    df_resultados = pd.DataFrame(resultados)
    return df_resultados, all_labels["PCA"], all_labels["UMAP"]

def busqueda_hdbscan_min_cluster(X, rango=(5, 50, 5), metricas=['silhouette', 'calinski', 'davies'], mostrar_tabla=True):
    """
    Evalúa múltiples configuraciones de `min_cluster_size` para HDBSCAN sobre los datos.

    Parámetros
    ----------
    X : np.ndarray
        Datos de entrada para clustering.
    rango : tuple, opcional
        Tupla que define el rango de búsqueda (inicio, fin, paso). Por defecto es (5, 50, 5).
    metricas : list of str, opcional
        Lista de métricas a calcular. Puede incluir 'silhouette', 'calinski', 'davies'. Por defecto es todas.
    mostrar_tabla : bool, opcional
        Si True, imprime la tabla de resultados. Por defecto es True.

    Retorna
    -------
    None
    """
    resultados = []
    for min_cluster in range(*rango):
        model = hdbscan.HDBSCAN(min_cluster_size=min_cluster, min_samples=4)
        labels = model.fit_predict(X)
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        if n_clusters <= 1:
            continue
        try:
            fila = {'min_cluster_size': min_cluster, 'n_clusters': n_clusters}
            if 'silhouette' in metricas:
                fila['silhouette'] = silhouette_score(X, labels)
            if 'calinski' in metricas:
                fila['calinski'] = calinski_harabasz_score(X, labels)
            if 'davies' in metricas:
                fila['davies'] = davies_bouldin_score(X, labels)
            resultados.append(fila)
        except:
            continue
    df = pd.DataFrame(resultados).sort_values(by="min_cluster_size")
    if mostrar_tabla and not df.empty:
        print("\nResultados HDBSCAN por min_cluster_size:")
        print(df.to_string(index=False))

def evaluar_hdbscan_clustering(X_pca, X_umap, min_samples_val=4, min_cluster_size_pca=5, min_cluster_size_umap=10):
    """
    Aplica HDBSCAN sobre datos reducidos con PCA y UMAP, y evalúa el rendimiento del clustering.

    Parámetros
    ----------
    X_pca : np.ndarray
        Datos transformados con PCA (2D).
    X_umap : np.ndarray
        Datos transformados con UMAP (2D).
    min_samples_val : int, opcional
        Valor de min_samples para HDBSCAN. Por defecto es 4.
    min_cluster_size_pca : int, opcional
        Tamaño mínimo de cluster para PCA. Por defecto es 5.
    min_cluster_size_umap : int, opcional
        Tamaño mínimo de cluster para UMAP. Por defecto es 10.

    Retorna
    -------
    df_resultados : pd.DataFrame
        Tabla con métricas de evaluación para cada técnica de reducción.
    labels_pca : np.ndarray
        Etiquetas resultantes del clustering sobre PCA.
    labels_umap : np.ndarray
        Etiquetas resultantes del clustering sobre UMAP.
    """
    resultados = []
    all_labels = {}

    for nombre, X, min_cluster_size in [("PCA", X_pca, min_cluster_size_pca), ("UMAP", X_umap, min_cluster_size_umap)]:
        model = hdbscan.HDBSCAN(min_samples=min_samples_val, min_cluster_size=min_cluster_size).fit(X)
        labels = model.labels_
        all_labels[nombre] = labels

        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

        if n_clusters >= 2:
            silhouette = silhouette_score(X, labels)
            ch_score = calinski_harabasz_score(X, labels)
            db_score = davies_bouldin_score(X, labels)
        else:
            silhouette = ch_score = db_score = float("nan")

        resultados.append({
            "Técnica": nombre,
            "Modelo": "HDBSCAN",
            "Clusters": n_clusters,
            "Silhouette": silhouette,
            "Calinski-Harabasz": ch_score,
            "Davies-Bouldin": db_score
        })

    df_resultados = pd.DataFrame(resultados)
    return df_resultados, all_labels["PCA"], all_labels["UMAP"]

**Bloque 3:** Detección de anomalías con Isolation Forest y One-Class SVM.

- **`optimizar_modelos_optuna()`** 
Optimiza hiperparámetros para Isolation Forest y One-Class SVM usando Optuna.

- **`entrenar_modelos()`** 
Entrena los modelos de detección de anomalías con los mejores parámetros encontrados.

---

##### `Decisiones de diseño`

##### **Justificación del uso de Optuna para la optimización de hiperparámetros:**

- La detección de anomalías con modelos como Isolation Forest y One-Class SVM depende fuertemente de ciertos hiperparámetros clave (como contamination, kernel, gamma, nu, entre otros). Una mala configuración puede llevar a resultados poco fiables, ya sea detectando muchas falsos positivos o ignorando puntos verdaderamente anómalos.

- Las ventajas específicas de usar Optuna en este contexto son:

    - Permite explorar de forma inteligente un espacio continuo o categórico de hiperparámetros.

    - Es mucho más eficiente que una búsqueda en cuadrícula (Grid Search) o aleatoria (Random Search).

    - Se adapta bien a funciones objetivo que no son fácilmente diferenciables ni convexas, como es el caso al optimizar modelos de detección de anomalías.

    - En este caso, la función objetivo fue maximizar la cantidad de puntos identificados como anómalos, bajo la hipótesis razonable de que los modelos iniciales subdetectaban anomalías relevantes.

- Gracias a Optuna, se lograron configuraciones robustas y eficientes de los modelos, sin necesidad de intervención manual ni ajustes arbitrarios.

##### **Justificación del uso de PCA y UMAP para la detección y comparación de anomalías:**

- Al igual que en el análisis de clustering, se aplicaron dos técnicas complementarias de reducción de dimensionalidad: PCA (lineal) y UMAP (no lineal). La razón para utilizarlas en el contexto de detección de anomalías es doble:

    - Reducción del ruido y la redundancia en los datos: Ambos métodos ayudan a mejorar la detección de outliers al comprimir la información más relevante en un espacio más manejable, facilitando la tarea del modelo.

    - Comparación de efectos estructurales:

        - PCA conserva principalmente la varianza global de los datos y es adecuada para estructuras lineales.

        - UMAP preserva la estructura local y la conectividad, siendo útil cuando hay relaciones no lineales más complejas o patrones de rareza más sutiles.

- Al aplicar los modelos en ambos espacios (PCA y UMAP), es posible comparar cómo varían los resultados de anomalías según la estructura subyacente capturada por cada técnica. Esto permite observar la estabilidad y sensibilidad del modelo ante distintas representaciones del mismo conjunto de datos, lo que enriquece el análisis y refuerza la validez de los outliers detectados si coinciden entre ambos enfoques.

---

In [ ]:
def optimizar_modelos_optuna(X_pca_final, X_umap_final, n_trials=30):
    """
    Optimiza los hiperparámetros de Isolation Forest y One-Class SVM utilizando Optuna.

    Para Isolation Forest, se busca el valor óptimo de `contamination` que maximice 
    la detección de anomalías en el espacio reducido por PCA.

    Para One-Class SVM, se ajustan los parámetros `kernel`, `gamma` y `nu` para 
    maximizar la detección de anomalías en el espacio reducido por UMAP.

    Parámetros:
    - X_pca_final (np.ndarray): Datos transformados por PCA.
    - X_umap_final (np.ndarray): Datos transformados por UMAP.
    - n_trials (int): Número de iteraciones de búsqueda por modelo.

    Retorna:
    - Tuple[dict, dict]: Los mejores hiperparámetros encontrados para Isolation Forest y One-Class SVM.
    """
    def objective_if(trial):
        contamination = trial.suggest_float("contamination", 0.01, 0.15)
        model = IsolationForest(contamination=contamination, random_state=42)
        model.fit(X_pca_final)
        preds = model.predict(X_pca_final)
        return -np.sum(preds == -1)

    def objective_ocsvm(trial):
        kernel = trial.suggest_categorical("kernel", ["rbf", "sigmoid", "poly"])
        gamma = trial.suggest_float("gamma", 1e-4, 1.0, log=True)
        nu = trial.suggest_float("nu", 0.01, 0.2)
        model = OneClassSVM(kernel=kernel, gamma=gamma, nu=nu)
        model.fit(X_umap_final)
        preds = model.predict(X_umap_final)
        return -np.sum(preds == -1)

    study_if = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
    study_if.optimize(objective_if, n_trials=n_trials)

    study_ocsvm = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
    study_ocsvm.optimize(objective_ocsvm, n_trials=n_trials)

    return study_if.best_params, study_ocsvm.best_params

def entrenar_modelos(X_pca_final, X_umap_final, params_if, params_ocsvm):
    """
    Entrena modelos de Isolation Forest y One-Class SVM con los mejores parámetros encontrados.

    Los modelos se entrenan tanto en el espacio PCA como UMAP, generando predicciones 
    de anomalías para cada combinación.

    Parámetros:
    - X_pca_final (np.ndarray): Datos transformados por PCA.
    - X_umap_final (np.ndarray): Datos transformados por UMAP.
    - params_if (dict): Hiperparámetros optimizados para Isolation Forest.
    - params_ocsvm (dict): Hiperparámetros optimizados para One-Class SVM.

    Retorna:
    - dict: Diccionario con predicciones de anomalías para cada modelo y técnica de reducción.
            Las claves son 'IF_PCA', 'IF_UMAP', 'OCSVM_PCA' y 'OCSVM_UMAP'.
    """
    modelos = {}

    modelo_if = IsolationForest(**params_if, random_state=42)
    modelos['IF_PCA'] = modelo_if.fit_predict(X_pca_final)
    modelos['IF_UMAP'] = modelo_if.fit_predict(X_umap_final)

    modelo_ocsvm = OneClassSVM(**params_ocsvm)
    modelos['OCSVM_PCA'] = modelo_ocsvm.fit(X_pca_final).predict(X_pca_final)
    modelos['OCSVM_UMAP'] = modelo_ocsvm.fit(X_umap_final).predict(X_umap_final)

    return modelos

**Bloque 4:** Funciones de visualización de gráficos y resultados de las funciones anteriores.

- **`visualizar_reduccion_2d()`** 
Visualiza en 2D las reducciones PCA y UMAP coloreando según la variable objetivo.

- **`graficar_cluster_comparado()`** 
Muestra visualmente los clusters obtenidos con PCA y UMAP para comparar resultados.

- **`graficar_anomalias_en_ax()`** 
Dibuja anomalías y puntos normales en un gráfico según etiquetas de detección.

- **`graficos_anomalias()`** 
Visualiza en conjunto las anomalías detectadas por Isolation Forest y One-Class SVM.

- **`tabla_anomalias()`** 
Genera una tabla con los índices de datos clasificados como anomalías por cada modelo.

- **`resumen_anomalias()`** 
Resume y cuenta las anomalías detectadas por cada modelo, destacando las detectadas por múltiples modelos.

---

##### `Decisiones de diseño`

##### **Justificación del análisis de anomalías:**

- Se realiza un análisis detallado de anomalías para evaluar la consistencia entre modelos y facilitar la interpretación. El resumen por modelo permite comparar la cantidad de anomalías detectadas por cada técnica, mientras que la identificación de observaciones repetidas entre modelos destaca aquellas potencialmente más confiables. Además, la tabla con los índices específicos de cada anomalía permite una trazabilidad clara y un análisis posterior más profundo.

---

In [ ]:
# Visualización de PCA y UMAP inicial (2 componentes)
def visualizar_reduccion_2d(X_pca, X_umap, y):
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=y, palette='coolwarm', ax=axes[0])
    axes[0].set_title("PCA")
    sns.scatterplot(x=X_umap[:, 0], y=X_umap[:, 1], hue=y, palette='coolwarm', ax=axes[1])
    axes[1].set_title("UMAP")
    plt.tight_layout()
    plt.show()

# Visualización de clusters con DBSCAN y HDBSCAN
def graficar_cluster_comparado(X_pca_2d, X_umap_2d, labels_pca, labels_umap, metodo="Clustering"):
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    sns.scatterplot(x=X_pca_2d[:, 0], y=X_pca_2d[:, 1], hue=labels_pca, palette="tab10", ax=axes[0], s=50, legend=False)
    axes[0].set_title(f"{metodo} PCA (2D)")
    axes[0].set_xlabel("Componente 1")
    axes[0].set_ylabel("Componente 2")
    axes[0].grid(True)
    sns.scatterplot(x=X_umap_2d[:, 0], y=X_umap_2d[:, 1], hue=labels_umap, palette="tab10", ax=axes[1], s=50, legend=False)
    axes[1].set_title(f"{metodo} UMAP (2D)")
    axes[1].set_xlabel("Componente 1")
    axes[1].set_ylabel("Componente 2")
    axes[1].grid(True)
    plt.suptitle(f"Clusters con {metodo}", fontsize=16)
    plt.tight_layout()
    plt.show()

# Visualización de anomalías
def graficar_anomalias_en_ax(ax, X_2d, labels, metodo, tecnica):
    sns.scatterplot(x=X_2d[:, 0], y=X_2d[:, 1],
                    hue=labels, palette={1: "grey", -1: "red"},
                    s=50, ax=ax, legend=False)
    ax.set_title(f"{metodo} ({tecnica})")
    ax.set_xlabel("Componente 1")
    ax.set_ylabel("Componente 2")
    ax.grid(True)

def graficos_anomalias(X_pca, X_umap, modelos):
    modelos_if = {k: v for k, v in modelos.items() if "IF" in k}
    modelos_ocsvm = {k: v for k, v in modelos.items() if "OCSVM" in k}

    fig1, axs1 = plt.subplots(1, 2, figsize=(12, 5))
    for ax, (metodo, labels) in zip(axs1, modelos_if.items()):
        tecnica = "PCA" if "PCA" in metodo else "UMAP"
        X_plot = X_pca if tecnica == "PCA" else X_umap
        graficar_anomalias_en_ax(ax, X_plot, labels, "Isolation Forest", tecnica)
    fig1.suptitle("Análisis de Anomalías - Isolation Forest", fontsize=14)
    plt.tight_layout()
    plt.show()

    fig2, axs2 = plt.subplots(1, 2, figsize=(12, 5))
    for ax, (metodo, labels) in zip(axs2, modelos_ocsvm.items()):
        tecnica = "PCA" if "PCA" in metodo else "UMAP"
        X_plot = X_pca if tecnica == "PCA" else X_umap
        graficar_anomalias_en_ax(ax, X_plot, labels, "One-Class SVM", tecnica)
    fig2.suptitle("Análisis de Anomalías - One-Class SVM", fontsize=14)
    plt.tight_layout()
    plt.show()

# Resumen de detección de anomalías
def tabla_anomalias(modelos):
    def extraer_indices(labels):
        return np.where(labels == -1)[0]

    data = []
    for nombre, labels in modelos.items():
        indices = extraer_indices(labels)
        for idx in indices:
            data.append({"Modelo": nombre, "Índice paciente": idx})

    df = pd.DataFrame(data)
    print("\nTabla de anomalías (formato largo):")
    print(df)
    return df

def resumen_anomalias(modelos):
    print("\nResumen de anomalías por modelo:")
    for nombre, labels in modelos.items():
        n_anom = np.sum(labels == -1)
        print(f"{nombre}: {n_anom} anomalías")

    indices_all = []
    for labels in modelos.values():
        indices_all.extend(np.where(labels == -1)[0])

    indices_counts = pd.Series(indices_all).value_counts()
    repetidos = indices_counts[indices_counts > 1]

    print("\nÍndices detectados como anomalías por más de un modelo:")
    if not repetidos.empty:
        print(repetidos)
    else:
        print("No hay índices repetidos entre modelos.")

    return indices_counts

**Bloque 5:** Función de ejecución.

- **`main()`** 
Ejecuta el pipeline completo de carga, reducción, clustering, optimización, detección y visualización.

---

In [ ]:
def main():
    """
    Flujo principal de ejecución del análisis de clustering y detección de anomalías en el dataset.

    Pasos realizados:
    1. Carga y preprocesamiento de datos (escalado).
    2. Aplicación de reducción de dimensionalidad con PCA y UMAP (tanto para visualización en 2D como para análisis con más componentes).
    3. Visualización inicial de los datos reducidos en 2D con colores según la variable objetivo (Outcome) para observar separación o agrupamiento potencial.
    4. Análisis de la curva k-distancia para determinar valores adecuados de epsilon en DBSCAN.
    5. Búsqueda y evaluación de parámetros para DBSCAN y HDBSCAN, incluyendo métricas de calidad de clustering.
    6. Visualización comparativa de clusters obtenidos con diferentes técnicas.
    7. Optimización de hiperparámetros para los modelos de detección de anomalías (Isolation Forest y One-Class SVM) usando Optuna.
    8. Entrenamiento y evaluación de modelos de detección de anomalías.
    9. Resumen y visualización de anomalías detectadas.

    Esta función orquesta el pipeline completo y muestra resultados intermedios para facilitar la interpretación.

    No recibe parámetros ni retorna valores, ya que está pensado para ejecución directa del script.
    """
    # Carga dedatos
    X_scaled, y = cargar_y_preprocesar()

    # Reducción de dimensionalidad
    X_pca, X_umap, X_pca_final, X_umap_final = aplicar_reduccion(X_scaled)
    visualizar_reduccion_2d(X_pca, X_umap, y)

    # Clustering
    # Búsqueda de eps para DBSCAN
    graficar_k_distance(X_pca_final, X_umap_final)
    eps_range_pca = [1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5]
    eps_range_umap = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1]
    resultados_dbscan_pca = busqueda_dbscan_eps(X_pca_final, eps_range_pca)
    resultados_dbscan_umap = busqueda_dbscan_eps(X_umap_final, eps_range_umap)

    # Búsqueda de min_cluster_size para HDBSCAN
    busqueda_hdbscan_min_cluster(X_pca_final)
    busqueda_hdbscan_min_cluster(X_umap_final)

    # Entrenamiento de modelos DBSCAN y HDBSCAN
    df_dbscan, labels_dbscan_pca, labels_dbscan_umap = evaluar_dbscan_clustering(X_pca_final, X_umap_final)
    df_hdbscan, labels_hdbscan_pca, labels_hdbscan_umap = evaluar_hdbscan_clustering(X_pca_final, X_umap_final)

    # Tabla comparativa de DBSCAN y HDBSCAN
    df_cluster_metrics = pd.concat([df_dbscan, df_hdbscan], ignore_index=True)
    display(df_cluster_metrics)

    # Gráficos de clusters encontrados con DBSCAN y HDBSCAN
    graficar_cluster_comparado(X_pca, X_umap, labels_dbscan_pca, labels_dbscan_umap, metodo="DBSCAN")
    graficar_cluster_comparado(X_pca, X_umap, labels_hdbscan_pca, labels_hdbscan_umap, metodo="HDBSCAN")

    # Detección de anomalías
    # Búsqueda de mejores hiperparámetros para Isolation Forest y One-Class SVM y entrenamiento de modelos con los mejores hiperparámetros encontrados
    params_if, params_ocsvm = optimizar_modelos_optuna(X_pca_final, X_umap_final)
    modelos = entrenar_modelos(X_pca_final, X_umap_final, params_if, params_ocsvm)

    # Resultados de anomalías por modelo, anomalías repetidas entre modelos y tabla de todas las anomalías encontradas en todos los modelos
    counts = resumen_anomalias(modelos)
    df_anomalias = tabla_anomalias(modelos)

    # Visualización de anomalías
    graficos_anomalias(X_pca_final, X_umap_final, modelos)

# 4. Visualización de resultados

Se muestran los resultados obtenidos a partir de la ejecución de la funcion **main()**.

---

In [ ]:
if __name__ == "__main__":
    main()

# 5. Análisis de los resultados y reflexiones finales

---

## Análisis Visual de Reducción de Dimensionalidad: PCA vs UMAP

Se aplicaron técnicas de reducción de dimensionalidad (PCA y UMAP) con dos componentes sobre los datos escalados, visualizando los puntos coloreados según la variable target (Outcome).

### Observaciones

- **Separación visual:**  
  Aunque los datos con target 0 tienden a agruparse en la parte inferior izquierda y los con target 1 hacia la parte superior derecha, no se observa una frontera clara ni clusters bien definidos. Esto es común cuando solo se usan dos componentes que no capturan toda la complejidad del dataset.

- **PCA:**  
  Método lineal que maximiza la varianza total, refleja la estructura global de los datos. Sin embargo, si las relaciones entre variables y target son no lineales o complejas, la separación entre clases es limitada, como se observa en la visualización.

- **UMAP:**  
  Técnica no lineal que preserva tanto la estructura local como global de los datos. Tiende a destacar agrupamientos más coherentes y patrones sutiles que PCA no logra captar completamente. En este caso, aunque la diferencia con PCA no es muy marcada, UMAP puede captar estructuras que PCA no.

- **Utilidad visual:**  
  La combinación de ambas visualizaciones permite contrastar la información estructural que cada técnica destaca. Visualizar con el target como color ayuda a evaluar el potencial de separación entre clases, indicando que 2 componentes son insuficientes para una separación clara.

### Conclusión

PCA y UMAP con dos componentes son útiles para una inspección inicial de la estructura de los datos y para justificar el uso posterior de técnicas de clustering y detección de anomalías, dado que la separación observada no es suficiente para clasificar con precisión solo con estas proyecciones.

---

## Análisis de Clustering: DBSCAN y HDBSCAN con PCA y UMAP

### Selección de Parámetros: k-distance, eps y min_cluster_size

Se utilizó la curva k-distance para estimar rangos adecuados del parámetro `eps` para DBSCAN en ambos espacios reducidos (PCA y UMAP). Esta técnica ayuda a identificar la "rodilla" en la curva que marca un buen valor de eps, evitando valores arbitrarios o erróneos que podrían llevar a clusters pobres o demasiados outliers.

Con base en estas curvas, se definieron rangos de eps para evaluar la calidad de los clusters con métricas como Silhouette y Davies-Bouldin.

Para HDBSCAN, se exploró un rango de valores para `min_cluster_size` con el fin de encontrar un tamaño mínimo óptimo de cluster. Esto es crucial porque un `min_cluster_size` muy pequeño o muy grande puede afectar la detección de clusters relevantes y la separación entre ellos.

---

### Resultados de Evaluación de eps para DBSCAN

- **PCA:**
  - Conforme aumenta eps de 1.8 a 2.4, el número de clusters se reduce de 3 a 2.
  - La métrica de Silhouette mejora gradualmente hasta 0.3875 con eps=2.4, indicando clusters más cohesionados.
  - El índice de Davies-Bouldin disminuye, mejorando la separación entre clusters.

- **UMAP:**
  - Para eps entre 0.7 y 0.9 se detectan 3 clusters con Silhouette moderada (~0.587).
  - Para eps = 1.0 y 1.1, el número de clusters baja a 2, pero con un aumento significativo en la Silhouette (0.688), y un Davies-Bouldin muy bajo (0.277), mostrando mejor calidad de clusters.

---

### Resultados de Evaluación de min_cluster_size para HDBSCAN

- **PCA:**
  - Min_cluster_size = 5 genera 3 clusters, con Silhouette de 0.275, CH moderado y Davies-Bouldin alto (2.48), lo que indica clusters menos definidos.
  - Para valores mayores (10-20), solo se detectan 2 clusters con métricas estables pero sin mejora significativa.

- **UMAP:**
  - Min_cluster_size = 5 genera 4 clusters con Silhouette alta (0.57) y CH muy alto, con Davies-Bouldin bajo (0.49), indicando clusters bien definidos.
  - Incrementar min_cluster_size reduce el número de clusters a 3 o 2, con valores similares o mejores en las métricas.

---

### Tabla Comparativa Final (eps seleccionados 2.4 para PCA y 1.0 para UMAP, min_cluster_size=5 para HDBSCAN)

| Técnica | Modelo  | Clusters | Silhouette | Calinski-Harabasz | Davies-Bouldin |
|---------|---------|----------|------------|-------------------|----------------|
| PCA     | DBSCAN  | 2        | 0.3875     | 24.21             | 1.98           |
| UMAP    | DBSCAN  | 2        | 0.6881     | 523.92            | 0.28           |
| PCA     | HDBSCAN | 3        | 0.2750     | 35.43             | 2.49           |
| UMAP    | HDBSCAN | 3        | 0.5874     | 1299.00           | 0.47           |

---

### Interpretación Visual y Calidad de Clusters

- **DBSCAN con PCA:**  
  Visualmente, se observa un gran grupo dominante (color naranja) y algunos puntos dispersos sin una separación clara en clusters, indicando que PCA con estos parámetros no facilita una segmentación precisa.

- **DBSCAN con UMAP:**  
  Se detectan dos clusters claramente definidos y separados, concordando con las métricas superiores (Silhouette y Davies-Bouldin). Esto sugiere que UMAP logra un espacio más adecuado para que DBSCAN identifique estructuras en los datos.

- **HDBSCAN con PCA:**  
  Similar a DBSCAN con PCA, los clusters no están bien diferenciados, predominando un gran grupo con pocos puntos aislados.

- **HDBSCAN con UMAP:**  
  Se observan tres clusters muy bien marcados, en línea con los altos valores de Silhouette y Calinski-Harabasz, indicando una agrupación más robusta.

---

### Conclusión de clustering

El análisis confirma que UMAP proporciona una representación del espacio más adecuada para aplicar algoritmos de clustering basados en densidad como DBSCAN y HDBSCAN en este dataset. Los parámetros seleccionados basados en las curvas k-distance y evaluación sistemática de eps y min_cluster_size permitieron obtener agrupamientos más coherentes y definidos en UMAP, mientras que PCA, aunque útil para reducción lineal, no logró una segmentación clara.

Este enfoque combinado de análisis visual, selección guiada de hiperparámetros y evaluación con múltiples métricas es clave para un clustering efectivo en datos complejos.

---

## Análisis de Detección de Anomalías

### Resultados Cuantitativos

- **Número de anomalías detectadas por modelo:**
  - Isolation Forest (PCA): 8 anomalías
  - Isolation Forest (UMAP): 8 anomalías
  - One-Class SVM (PCA): 12 anomalías
  - One-Class SVM (UMAP): 8 anomalías

- **Anomalías detectadas por más de un modelo:**
  - Algunos índices se repiten en al menos dos modelos, destacando especialmente los índices 706, 494, 81 y 60, detectados por tres modelos, lo que sugiere una alta probabilidad de ser casos realmente atípicos.

- **Distribución de anomalías:**
  - La tabla de anomalías muestra qué modelo detectó cada índice, evidenciando coincidencias y discrepancias en la detección.

---

### Interpretación Visual y Comportamiento

- Los gráficos revelaron que todos los modelos detectaron un pequeño grupo de puntos como outliers, mostrando consistencia en las anomalías identificadas.

- **Diferencias según la reducción usada:**
  - Los modelos entrenados con PCA presentan anomalías dispersas alrededor de la nube principal de datos, sugiriendo que el espacio lineal generado por PCA puede generar detecciones más distribuidas.
  - En contraste, los modelos basados en UMAP detectaron anomalías agrupadas en un pequeño conjunto de puntos cercanos entre sí, lo que indica que UMAP, con su enfoque no lineal, puede agrupar mejor las anomalías relacionadas o estructuradas de manera similar.

---

### Consideraciones

El uso de hiperparámetros optimizados mediante Optuna asegura que cada modelo esté ajustado para maximizar la detección eficiente y precisa de anomalías, lo que aporta confianza a los resultados.

La coincidencia de anomalías en múltiples modelos refuerza la validez de esos puntos como casos atípicos relevantes, mientras que las diferencias resaltan cómo la elección del espacio de características (PCA vs UMAP) afecta la forma y concentración de la detección.

En conjunto, este análisis sugiere que, para este dataset, la combinación de reducción no lineal (UMAP) con modelos de detección de anomalías puede ofrecer una identificación más focalizada y estructurada de puntos atípicos.

---

## Análisis Cruzado y Conclusiones Finales

### ¿Coinciden las anomalías con clústeres raros?

- Al realizar una comparación cruzada entre los resultados de clustering y los de detección de anomalías, se observa una relación significativa entre las anomalías detectadas y la presencia de clústeres poco poblados o alejados del resto (es decir, clústeres raros). Este patrón se aprecia con mayor claridad en los modelos entrenados sobre las proyecciones de UMAP, donde tanto DBSCAN como HDBSCAN logran identificar clústeres más compactos y bien definidos, y donde las anomalías tienden a agruparse fuera de estas regiones densas. De hecho, en UMAP los puntos detectados como outliers por múltiples modelos coinciden espacialmente con zonas alejadas de los clústeres principales o con regiones donde los algoritmos de clustering no asignan etiqueta de clúster (etiqueta -1 en DBSCAN).

- En cambio, al observar los resultados sobre las proyecciones de PCA, la relación entre anomalías y clústeres raros es menos evidente. En PCA, las anomalías tienden a estar más dispersas, rodeando la nube principal de datos sin formar necesariamente agrupaciones visibles. Esto sugiere que, al menos en este caso, PCA conserva suficiente varianza para describir la estructura general del dataset, pero no logra capturar con la misma efectividad las estructuras locales o no lineales donde las anomalías podrían manifestarse de forma más clara.

- Además, al observar los índices detectados repetidamente como anómalos por múltiples modelos, se confirma que ciertos casos se consideran consistentes outliers independientemente del algoritmo o la proyección, lo que refuerza su carácter de verdaderas anomalías dentro del conjunto de datos. Varios de estos puntos coinciden también con clústeres pequeños o aislados, lo que apoya aún más la hipótesis de que los clústeres raros y las anomalías no son fenómenos completamente disjuntos.

---

### ¿Qué técnica dio resultados más interpretables?

- En términos de interpretabilidad, la reducción de dimensionalidad mediante **UMAP** fue claramente más efectiva que PCA. Las proyecciones 2D obtenidas con UMAP ofrecieron una separación más clara entre diferentes regiones del espacio de datos, permitiendo a simple vista distinguir clústeres densos, zonas transicionales y regiones aisladas con pocos puntos. Esto facilitó tanto la evaluación visual del clustering como la localización de potenciales outliers.

- La combinación de UMAP con HDBSCAN, en particular, resultó muy útil para identificar estructuras internas complejas en los datos, incluyendo clústeres de distintos tamaños y densidades. Además, al aplicar modelos de detección de anomalías sobre las proyecciones de UMAP, se observaron patrones más definidos: los outliers se concentraban en zonas específicas, y no simplemente como ruido disperso. Esto no solo ayudó a validar visualmente los resultados, sino que también aportó una narrativa más coherente sobre la estructura interna del dataset.

- Por otro lado, PCA, si bien útil para mantener la varianza global, presentó visualizaciones más difusas, con menos capacidad para distinguir agrupaciones no lineales. Las anomalías detectadas en PCA tendieron a ubicarse en los bordes del espacio proyectado, sin una organización clara ni agrupamientos visuales, lo que dificulta su interpretación sin herramientas adicionales.

---

### Conclusión general

- El análisis cruzado sugiere que la detección de anomalías y el clustering no deben tratarse como procesos completamente separados: existe una relación evidente entre clústeres raros y anomalías, especialmente cuando se utilizan técnicas de reducción de dimensionalidad no lineales como UMAP que preservan la estructura local del espacio. Los modelos de detección de anomalías aplicados a proyecciones de UMAP fueron más consistentes y más fáciles de interpretar visualmente, revelando agrupaciones de outliers que serían difíciles de detectar en el espacio original o bajo PCA.

- En conjunto, los resultados destacan la importancia de aplicar técnicas de reducción de dimensionalidad adecuadas antes de realizar clustering o detección de outliers, y de considerar análisis visuales y cuantitativos de manera conjunta para obtener conclusiones robustas. La sinergia entre UMAP, HDBSCAN e Isolation Forest / One-Class SVM demostró ser una combinación potente para el análisis exploratorio y la identificación de patrones atípicos en datos complejos.

### Recomendaciones Finales

- Usar UMAP como paso previo a clustering o detección de anomalías permite identificar estructuras no lineales y clústeres atípicos con mayor claridad.
- Para datasets donde las clases o anomalías no están bien separadas linealmente, se recomienda combinar UMAP + HDBSCAN + Isolation Forest u OCSVM.
- Visualizar los datos proyectados en 2D con el target como color puede ayudar a detectar si hay señales útiles en los datos antes de aplicar modelos supervisados.
- Incorporar técnicas de validación cruzada de anomalías, como contar los casos detectados por múltiples métodos, mejora la confianza en los resultados.

---